In [12]:
from pathlib import Path
from matplotlib import pyplot as plt
import pickle
from collections import defaultdict
import os
from inspect import signature

import dimelo
from dimelo import parse_bam, load_processed
from dimelo.test import RelativePath,DiMeLoParsingTestCase,filter_kwargs_for_func

# Base input and output directories
test_data_dir = Path('./data')
output_dir = test_data_dir / 'test_targets'

output_dir.mkdir(exist_ok=True)

region = 'chr14:17376348-17378348'

# Paths to input files
ctcf_bam_file = test_data_dir / 'ctcf_demo.sorted.bam'
# ctcf_guppy_bam_file = test_data_dir / 'winnowmap_guppy_merge_subset.updated.bam'
ctcf_target_regions = RelativePath(test_data_dir / 'ctcf_demo_peak.bed')
ctcf_off_target_regions = RelativePath(test_data_dir / 'ctcf_demo_not_peak.bed')
ref_genome_file = Path('./output/chm13.draft_v1.0.fasta')
ctcf_bam_file_updated =  RelativePath('./output/ctcf_demo.updated.bam')
output_dir = RelativePath(output_dir)

# Set up input files
DiMeLoParsingTestCase.setup_class()

Reference genome already downloaded.
Input bam already retagged.


## Create or load test matrix

### Load a pre-existing test matrix

This code should be used if you are trying to update only some test cases. Tests can only be updated by category in rest of the code. Only run categories where you are confident that you haven't broken any functionality.

In [13]:
if Path(RelativePath('data/test_targets/test_matrix.pickle')).exists():
    with open(RelativePath('data/test_targets/test_matrix.pickle'), 'rb') as file:
        test_matrix = pickle.load(file)

### Create a fresh test matrix

This code should be used if you are re-creating your test matrix from scratch: you should know everything is working correctly and plan to run all the remaining cells in the notebook to get all the different test cases covered.

In [14]:
test_matrix = {
#     'megalodon_peaks_190':(
#         # input kwargs
#         {
#             'input_file':ctcf_bam_file_updated,
#             'output_name':'megalodon_peaks_190',
#             'output_directory':output_dir,
#             'regions':[ctcf_target_regions,ctcf_off_target_regions],
#             'motifs':['A,0','CG,0'],
#             'thresh':190,
#             'window_size':1000,
#             'sort_by':['chromosome','read_start','read_name'],
#             'smooth_window':1,
#             'title':'megalodon_peaks_190',
#         },
#         # outputs dict function:values
#         {}, # populated in subsequent cells
#     ),
    'megalodon_single_190':(
        # input kwargs
        {
            'input_file':ctcf_bam_file_updated,
            'output_name':'megalodon_single_190',
            'output_directory':output_dir,
            'regions':region,
            'motifs':['A,0','CG,0'],
            'thresh':190,
            'window_size':None,
            'sort_by':['chromosome','read_start','read_name'],
            'smooth_window':10,
            'title':'megalodon_single_190',
        },
        # outputs dict function:values
        {}, # populated in subsequent cells
    ),
#     'megalodon_single_and_peaks_190':(
#         # input kwargs
#         {
#             'input_file':ctcf_bam_file_updated,
#             'output_name':'megalodon_single_and_peaks_190',
#             'output_directory':output_dir,
#             'regions':[region,ctcf_target_regions,ctcf_off_target_regions],
#             'motifs':['A,0','CG,0'],
#             'thresh':190,
#             'window_size':1000,
#             'sort_by':['chromosome','read_start','read_name'],
#             'smooth_window':100,
#             'title':'megalodon_single_and_peaks_190',
#         },
#         # outputs dict function:values
#         {}, # populated in subsequent cells
#     ),
#     'megalodon_single_nothresh':(
#         # input kwargs
#         {
#             'input_file':ctcf_bam_file_updated,
#             'output_name':'megalodon_single_nothresh',
#             'output_directory':output_dir,
#             'regions':region,
#             'motifs':['A,0','CG,0'],
#             'thresh':None,
#             'window_size':None,
#             'sort_by':['chromosome','read_end','read_name'],
#             'smooth_window':1,
#             'title':'megalodon_single_nothresh',
#         },
#         # outputs dict function:values
#         {}, # populated in subsequent cells
#     )
}

## Generate parse_bam outputs

### pileup

In [15]:
for kwargs,results in test_matrix.values():
    kwargs_pileup = filter_kwargs_for_func(parse_bam.pileup,kwargs)
    pileup_file, pileup_regions = parse_bam.pileup(
        **kwargs_pileup,
        ref_genome = ref_genome_file,
    )
    results['pileup'] = (RelativePath(pileup_file),RelativePath(pileup_regions))

# pileup_file, pileup_regions = parse_bam.pileup(
#     input_file=ctcf_bam_file_updated,
#     output_name='megalodon_merged_regions',
#     ref_genome=ref_genome_file,
#     output_directory=output_dir,
#     regions=[ctcf_target_regions,ctcf_off_target_regions],
#     motifs=['A,0','CG,0'],
#     thresh=190,
#     window_size=1000,
# )
# pileup_file_one, pileup_regions_one = parse_bam.pileup(
#     input_file=ctcf_bam_file_updated,
#     output_name='megalodon_one_region',
#     ref_genome=ref_genome_file,
#     output_directory=output_dir,
#     regions=region,
#     motifs=['A,0','CG,0'],
#     thresh=190,
#     window_size=None,
# )

No specified number of cores requested. 32 available on machine, allocating all.
Modification threshold of 190 assumed to be for range 0-255. 190/255=0.7450980392156863 will be sent to modkit.


          | Preprocessing   0% | 00:00

          | Processing ctcf_demo.updated.bam   0% | 00:00<?

          |    0%

### extract

In [16]:
for kwargs,results in test_matrix.values():
    kwargs_extract = filter_kwargs_for_func(parse_bam.extract,kwargs)
    if kwargs['regions']==region: # for now, we only want to extract with the single region due to output file size
        extract_file, extract_regions = parse_bam.extract(
            **kwargs_extract,
            ref_genome = ref_genome_file,
        )
        results['extract'] = (RelativePath(extract_file),RelativePath(extract_regions))
    else:
        results['extract'] = (None,None)

No specified number of cores requested. 32 available on machine, allocating all.
Modification threshold of 190 assumed to be for range 0-255. 190/255=0.7450980392156863 will be sent to modkit.


          | Preprocessing   0% | 00:00

          | Processing ctcf_demo.updated.bam   0% | 00:00<?

          |    0%

          | Transferring 24 from reads.A,0.txt into reads.combined_basemods.h5, new size 24   0% | 00:00<?

          | Preprocessing   0% | 00:00

          | Processing ctcf_demo.updated.bam   0% | 00:00<?

          |    0%

          | Transferring 24 from reads.CG,0.txt into reads.combined_basemods.h5, new size 48   0% | 00:00<?

## Generate load_processed outputs

### pileup_counts_from_bedmethyl

In [17]:
for kwargs,results in test_matrix.values():
    results['pileup_counts_from_bedmethyl'] = {}
    kwargs_func = filter_kwargs_for_func(load_processed.pileup_counts_from_bedmethyl,kwargs)
    for motif in kwargs['motifs']:
        results['pileup_counts_from_bedmethyl'][motif] = load_processed.pileup_counts_from_bedmethyl(
            bedmethyl_file = results['pileup'][0],
            **kwargs_func,
            motif = motif,
        )

### pileup_vectors_from_bedmethyl

In [18]:
for kwargs,results in test_matrix.values():
    results['pileup_vectors_from_bedmethyl'] = {}
    kwargs_func = filter_kwargs_for_func(load_processed.pileup_vectors_from_bedmethyl,kwargs)
    for motif in kwargs['motifs']:
        results['pileup_vectors_from_bedmethyl'][motif] = load_processed.pileup_vectors_from_bedmethyl(
            bedmethyl_file = results['pileup'][0],
            **kwargs_func,
            motif=motif,
        )

### read_vectors_from_hdf5

In [19]:
for kwargs,results in test_matrix.values():
    extract_file,regions_bed = results['extract']
    if extract_file is not None and regions_bed is not None:
        kwargs_func = filter_kwargs_for_func(load_processed.read_vectors_from_hdf5,kwargs)
        read_data_list, datasets, _ = load_processed.read_vectors_from_hdf5(
            file=extract_file,
            **kwargs_func,
        )        
        read_data_dict = {}
        # Pull out the data from the first read
        for idx,dataset in enumerate(datasets):
            for read_data in read_data_list:
                read_data_dict[dataset] = read_data[idx]
                break    
        results['read_vectors_from_hdf5'] = read_data_dict

## Save text matrix

In [20]:
print(test_matrix)

{'megalodon_single_190': ({'input_file': <dimelo.test.RelativePath object at 0x2aedbdb93090>, 'output_name': 'megalodon_single_190', 'output_directory': <dimelo.test.RelativePath object at 0x2aedbd9dfb10>, 'regions': 'chr14:17376348-17378348', 'motifs': ['A,0', 'CG,0'], 'thresh': 190, 'window_size': None, 'sort_by': ['chromosome', 'read_start', 'read_name'], 'smooth_window': 10, 'title': 'megalodon_single_190'}, {'pileup': (<dimelo.test.RelativePath object at 0x2aedbbbc3350>, <dimelo.test.RelativePath object at 0x2aedbd7f5dd0>), 'extract': (<dimelo.test.RelativePath object at 0x2aedbda81990>, <dimelo.test.RelativePath object at 0x2aedbd8aed90>), 'pileup_counts_from_bedmethyl': {'A,0': (82, 9803), 'CG,0': (96, 1129)}, 'pileup_vectors_from_bedmethyl': {'A,0': (array([0, 0, 0, ..., 0, 0, 0]), array([ 0,  0,  8, ...,  9,  0, 15])), 'CG,0': (array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]))}, 'read_vectors_from_hdf5': {'chromosome': 'chr14', 'mod_vector': array([False, False, 

In [21]:
with open('./data/test_targets/test_matrix.pickle', 'wb') as file:
    pickle.dump(test_matrix, file)

In [22]:
# counts_dict = defaultdict(dict)
# vectors_dict = defaultdict(dict)
# binarized_reads_dict = defaultdict(dict)
# prob_reads_dict = defaultdict(dict)
# for motif in ['A,0','CG,0']:
#     # Extract the total counts for the motif/regions
#     for regions in [region,ctcf_target_regions]:
#         counts_dict[motif][regions] = load_processed.pileup_counts_from_bedmethyl(
#             bedmethyl_file = pileup_file,
#             motif = motif,
#             regions = regions
#         )
#     # Extract counts profiles for the motif/regions
#     vectors_dict[motif][regions] = load_processed.pileup_vectors_from_bedmethyl(
#         bedmethyl_file = pileup_file,
#         motif = motif,
#         regions = regions,
#         window_size = 10, # Trim/extend regions to same size    
#     )
#     # Extract binarized read vectors for the motif/regions
#     read_data_list, datasets, _ = load_processed.read_vectors_from_hdf5(
#         file=extract_file, # binarized modification calls
#         regions=regions,
#         motifs=[motif],
#         sort_by = ['chromosome','read_start','read_name']
#     )
#     read_data_dict = {}
#     for idx,dataset in enumerate(datasets):
#         for read_data in read_data_list:
#             read_data_dict[dataset] = read_data[idx]
#             break    
#     binarized_reads_dict[motif][regions] = read_data_dict
#     # Extract probability read vectors for the motif/regions
#     read_data_list, datasets, _ = load_processed.read_vectors_from_hdf5(
#         file=extract_file_no_thresh, # raw modification probabilities
#         regions=regions,
#         motifs=[motif],
#         sort_by = ['chromosome','read_start','read_name']
#     )
#     read_data_dict = {}
#     # Print out the data from the first read
#     for idx,dataset in enumerate(datasets):
#         for read_data in read_data_list:
#             read_data_dict[dataset] = read_data[idx]
#             break
#     prob_reads_dict[motif][regions] = read_data_dict
# data_struct = (counts_dict,vectors_dict,binarized_reads_dict,prob_reads_dict)
# # Pickle the combined structure to a file
# with open(target_paths_dict['load_processed'], 'wb') as file:
#     pickle.dump(data_struct, file)